In [ ]:
# Cell 1
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# Cell 2
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Cell 2a
# Load the dataset
train_data = pd.read_csv(train_file_path, sep="\t", header=None, names=["label", "message"])
test_data = pd.read_csv(test_file_path, sep="\t", header=None, names=["label", "message"])

# Display the first few rows
print("Training Data:\n", train_data.head(), "\n")
print("Test Data:\n", test_data.head(), "\n")

# Check for missing values
print("Missing values in training data:\n", train_data.isnull().sum(), "\n")
print("Missing values in test data:\n", test_data.isnull().sum(), "\n")

# Check class distribution
print("Class distribution in training data:\n", train_data['label'].value_counts(), "\n")
print("Class distribution in test data:\n", test_data['label'].value_counts(), "\n")

# Convert labels to numerical values (spam=1, ham=0)
train_data['label'] = train_data['label'].map({'ham': 0, 'spam': 1})
test_data['label'] = test_data['label'].map({'ham': 0, 'spam': 1})


In [ ]:
# Cell 2b
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenization parameters
vocab_size = 5000  # Number of unique words to keep
max_length = 100  # Max words in a message
trunc_type = 'post'  # Truncate at the end if too long
oov_token = "<OOV>"  # Token for unknown words

# Initialize tokenizer and fit on training text
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_data['message'])

# Convert text to sequences (numerical representation)
train_sequences = tokenizer.texts_to_sequences(train_data['message'])
test_sequences = tokenizer.texts_to_sequences(test_data['message'])

# Pad sequences to ensure uniform length
train_padded = pad_sequences(train_sequences, maxlen=max_length, truncating=trunc_type)
test_padded = pad_sequences(test_sequences, maxlen=max_length, truncating=trunc_type)

# Convert labels to NumPy arrays
train_labels = np.array(train_data['label'])
test_labels = np.array(test_data['label'])

# Print example of tokenized and padded message
print("Original message:", train_data['message'].iloc[0])
print("Tokenized sequence:", train_sequences[0])
print("Padded sequence:", train_padded[0])


In [ ]:
# Cell 2c
# Build the neural network model
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, 64, input_length=max_length),  # Word embedding layer
    keras.layers.Bidirectional(keras.layers.LSTM(64)),  # LSTM for sequence learning
    keras.layers.Dense(32, activation='relu'),  # Fully connected layer
    keras.layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels), verbose=1)

# Display training summary
print("\nTraining complete!")


In [ ]:
# Cell 3
# Function to predict messages based on model
# (should return a list containing prediction probability and label, e.g., [0.0083, 'ham'])

def predict_message(pred_text):
    # Convert text to sequence
    sequence = tokenizer.texts_to_sequences([pred_text])

    # Pad the sequence to match model input shape
    padded_sequence = pad_sequences(sequence, maxlen=max_length, truncating=trunc_type)

    # Predict spam probability
    prediction = model.predict(padded_sequence)[0][0]

    # Assign label based on threshold (0.5)
    label = "spam" if prediction >= 0.5 else "ham"

    return [float(prediction), label]

# Test the function
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


In [ ]:
# Cell 4
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
